In [27]:
# Gerekli kütüphaneleri import ediyoruz
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval

# Otel yorumları veri setini yüklüyoruz
data = pd.read_csv(r"C:\Users\sulen\OneDrive\Masaüstü\Python ile Veri Analizi Atölyesi\Hotel_Reviews.csv")

# Veri setinin ilk 5 satırını görüntüleyerek yapıyı inceliyoruz
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


Bu veri seti, çeşitli otellere yapılan müşteri yorumlarını ve otel ile ilgili bazı bilgileri içermektedir. Her satır, tek bir yorum kaydını temsil etmektedir ve sütunlar arasında otelin adresi, adı, ortalama puanı, yorum tarihi, yorum yapan kişinin milliyeti ve olumsuz/olumlu yorum metinleri bulunmaktadır. Ayrıca yorumların toplam kelime sayıları, yorum sayıları, değerlendirme puanları ve yorum yapan kişinin önceki yorum sayıları gibi ek bilgiler de yer almaktadır. Bu yapı, otellerin müşteri geri bildirimlerini analiz etmek, yorum metinlerinden duygu analizi yapmak ve bir otel öneri sistemi geliştirmek için uygun bir temel sunmaktadır.

In [28]:
# 'United Kingdom' ifadesini 'UK' ile değiştiriyoruz (daha kısa ve standart isimlendirme için)
data.Hotel_Address = data.Hotel_Address.str.replace("United Kingdom", "UK")

# Hotel_Address sütununu boşluklardan ayırarak, son kelimeyi alıyoruz ve ülke sütunu oluşturuyoruz
data["countries"] = data.Hotel_Address.apply(lambda x: x.split(' ')[-1])

# Veri setindeki benzersiz ülkeleri kontrol ediyoruz
print(data.countries.unique())

['Netherlands' 'UK' 'France' 'Spain' 'Italy' 'Austria']


Veri setindeki otel adresleri üzerinde düzenleme yapılmıştır. Öncelikle “United Kingdom” ifadesi daha kısa ve standart bir format için “UK” olarak değiştirilmiştir. Ardından otel adresleri boşluklardan ayrılarak, adresin son kelimesi alınmış ve bu şekilde her otelin hangi ülkede bulunduğunu gösteren yeni bir “countries” sütunu oluşturulmuştur. Elde edilen benzersiz ülke değerleri, veri setinde otellerin Hollanda, Birleşik Krallık (UK), Fransa, İspanya, İtalya ve Avusturya’da konumlandığını göstermektedir.

In [29]:
# Analiz için gereksiz veya kullanılmayacak sütunları veri setinden çıkarıyoruz
data.drop(['Additional_Number_of_Scoring', 
           'Review_Date','Reviewer_Nationality','Negative_Review',
           'Review_Total_Negative_Word_Counts',
           'Total_Number_of_Reviews', 'Positive_Review',
           'Review_Total_Positive_Word_Counts',
           'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
           'days_since_review', 'lat', 'lng'], axis=1, inplace=True, errors='ignore')

# Kalan sütunları kontrol ediyoruz
print(data.columns)

Index(['Hotel_Address', 'Average_Score', 'Hotel_Name', 'Tags', 'countries'], dtype='object')


Veri setindeki analiz için gerekli olmayan sütunlar temizlenmiştir. Bu sütunlar arasında yorum tarihi, yorum yapan kişinin milliyeti, olumsuz ve olumlu yorum metinleri, toplam kelime sayıları, toplam yorum sayıları, yorum yapan kişinin toplam yorum sayısı, bireysel puanlar, tarihsel gün sayısı ve koordinatlar yer almaktadır. Bu işlem sonucunda veri setinde yalnızca otel adresi (Hotel_Address), ortalama puan (Average_Score), otel adı (Hotel_Name), etiketler (Tags) ve ülke bilgisi (countries) sütunları kalmıştır. Bu sadeleştirilmiş yapı, otel öneri sistemi oluşturmak için gerekli temel bilgileri sağlamaktadır.

In [30]:
# 'Tags' sütunundaki verileri string olarak düzeltme
def impute(column):
    # Eğer liste değilse direkt string olarak al
    if isinstance(column, list):
        return " ".join(column)
    else:
        return str(column)

# Fonksiyonu Tags sütununa uygula
data["Tags"] = data["Tags"].apply(impute)

# Sonucu kontrol et
data.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Family with young childre...",Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",Netherlands


Veri setindeki analiz için gerekli olmayan sütunlar temizlenmiştir. Bu sütunlar arasında yorum tarihi, yorum yapan kişinin milliyeti, olumsuz ve olumlu yorum metinleri, toplam kelime sayıları, toplam yorum sayıları, yorum yapan kişinin toplam yorum sayısı, bireysel puanlar, tarihsel gün sayısı ve koordinatlar yer almaktadır. Bu işlem sonucunda veri setinde yalnızca otel adresi (Hotel_Address), ortalama puan (Average_Score), otel adı (Hotel_Name), etiketler (Tags) ve ülke bilgisi (countries) sütunları kalmıştır. Bu sadeleştirilmiş yapı, otel öneri sistemi oluşturmak için gerekli temel bilgileri sağlamaktadır.

In [31]:
# Ülke ve etiket sütunlarını küçük harfe çevir
data['countries'] = data['countries'].str.lower()
data['Tags'] = data['Tags'].str.lower()

Bu işlem, “countries” ve “Tags” sütunlarındaki tüm değerleri küçük harfe çevirir. Amaç, analiz sırasında büyük/küçük harf farklılıklarından kaynaklanabilecek tutarsızlıkları ortadan kaldırmaktır. Örneğin “UK” ile “uk” veya “Leisure Trip” ile “leisure trip” artık aynı değer olarak kabul edilir ve metin madenciliği, filtreleme veya görselleştirme işlemlerinde doğru şekilde işlenir.

In [33]:
def recommend_hotel(location, description):
    # Kullanıcının açıklamasını küçük harfe çevir
    description = description.lower()
    
    # Açıklamayı kelimelere ayır
    word_tokenize(description)
    
    # İngilizce stopword listesi ve lemmatizer
    stop_words = stopwords.words('english')
    lemm = WordNetLemmatizer() 
    
    # Stopword'leri çıkar ve kelimeleri lemmatize et
    filtered = {word for word in description if not word in stop_words}
    filtered_set = set()
    for fs in filtered:
        filtered_set.add(lemm.lemmatize(fs))
    
    # Kullanıcının seçtiği ülkeye ait otelleri seç
    country = data[data['countries'] == location.lower()]
    country = country.set_index(np.arange(country.shape[0]))
    
    # Benzerlik hesaplaması için boş listeler
    cos = []
    
    # Her otel için kullanıcı açıklamasıyla Tags karşılaştırması
    for i in range(country.shape[0]):
        temp_token = word_tokenize(country["Tags"][i])
        temp_set = [word for word in temp_token if not word in stop_words]
        temp2_set = set()
        for s in temp_set:
            temp2_set.add(lemm.lemmatize(s))
        
        # Ortak kelime sayısını benzerlik olarak kaydet
        vector = temp2_set.intersection(filtered_set)
        cos.append(len(vector))
    
    # Benzerlik skorunu dataframe'e ekle
    country['similarity'] = cos
    
    # Benzerliğe göre sırala ve otelleri eşsiz hâle getir
    country = country.sort_values(by='similarity', ascending=False)
    country.drop_duplicates(subset='Hotel_Name', keep='first', inplace=True)
    
    # Ortalama puana göre yeniden sırala
    country.sort_values('Average_Score', ascending=False, inplace=True)
    country.reset_index(inplace=True)
    
    # Önerilen ilk 5 oteli döndür
    return country[["Hotel_Name", "Average_Score", "Hotel_Address"]].head()

Bu fonksiyon, kullanıcının belirttiği konum ve açıklamaya göre otel önerisi yapmaktadır. Fonksiyon önce açıklamayı kelimelere ayırır, stopword’leri çıkarır ve kelimeleri kök hâline getirir (lemmatize). Ardından seçilen ülkeye ait otellerin “Tags” sütunlarıyla karşılaştırma yapar ve ortak kelime sayısını benzerlik skoru olarak kaydeder. Oteller benzerliğe göre sıralanır, aynı isimli otellerden yalnızca biri tutulur ve ardından ortalama puana göre yeniden sıralanır. Fonksiyon, en uygun ilk 5 oteli “Hotel_Name”, “Average_Score” ve “Hotel_Address” bilgileriyle döndürür.

In [37]:
import nltk  
# nltk kütüphanesini projeye dahil eder

nltk.data.path.append(r"C:\Users\sulen\OneDrive\Masaüstü\Python ile Veri Analizi Atölyesi\nltk_data")  
# nltk_data klasörünün yolunu programa tanıtır

from nltk.corpus import stopwords  
# nltk içindeki stopwords (gereksiz kelimeler) listesini kullanmak için import eder

In [38]:
 recommend_hotel('Italy', 'I am going for a business trip') #sorgu

,Hotel_Name,Average_Score,Hotel_Address
0,Excelsior Hotel Gallia Luxury Collection Hotel,9.4,Piazza Duca D Aosta 9 Central Station 20124 Mi...
1,Palazzo Parigi Hotel Grand Spa Milano,9.3,Corso Di Porta Nuova 1 Milan City Center 20121...
2,Hotel Spadari Al Duomo,9.3,Via Spadari 11 Milan City Center 20123 Milan I...
3,Room Mate Giulia,9.3,Silvio Pellico 4 Milan City Center 20121 Milan...
4,UNA Maison Milano,9.3,Via Mazzini 4 Milan City Center 20123 Milan Italy


Sorgu sonucunda Milano’daki merkezi ve yüksek puanlı oteller önerilmiştir. Özellikle Excelsior Hotel Gallia gibi iş seyahatine uygun, ulaşım kolaylığı sağlayan oteller öne çıkmaktadır.

In [39]:
recommend_hotel('UK','I am going on a honeymoon, I need a honeymoon suite room for 3 nights') #sorgu

,Hotel_Name,Average_Score,Hotel_Address
0,Haymarket Hotel,9.6,1 Suffolk Place Westminster Borough London SW1...
1,41,9.6,41 Buckingham Palace Road Westminster Borough ...
2,Ham Yard Hotel,9.5,One Ham Yard Westminster Borough London W1D 7D...
3,Milestone Hotel Kensington,9.5,1 Kensington Court Kensington and Chelsea Lond...
4,Taj 51 Buckingham Gate Suites and Residences,9.5,Buckingham Gate Westminster Borough London SW1...
